Install Dependencies

In [55]:
# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

#Imports pickle
import pickle

#Library for indicators
from finta import TA

#Another library for indicators
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.trend import PSARIndicator
from ta.trend import SMAIndicator
from ta.trend import macd
from ta.momentum import rsi
from ta.volume import OnBalanceVolumeIndicator

#Import yfinance
import yfinance as yf

In [56]:
# First import the libraries that we need to use
import pandas as pd
import requests
import json

def fetch_daily_data(symbol):
    pair_split = symbol.split('/')  # symbol must be in format XXX/XXX ie. BTC/EUR
    symbol = pair_split[0] + '-' + pair_split[1]
    url = f'https://api.pro.coinbase.com/products/{symbol}/candles?granularity=86400'
    response = requests.get(url)
    if response.status_code == 200:  # check to make sure the response from server is good
        data = pd.DataFrame(json.loads(response.text), columns=['unix', 'low', 'high', 'open', 'close', 'volume'])
        data['date'] = pd.to_datetime(data['unix'], unit='s')  # convert to a readable date
        data['vol_fiat'] = data['volume'] * data['close']      # multiply the BTC volume by closing price to approximate fiat volume

# if we failed to get any data, print an error...otherwise write the file
         if data is None:
             print("Did not return any data from Coinbase for this symbol")
         else:
             data.to_csv(f'Coinbase_{pair_split[0] + pair_split[1]}_dailydata.csv', index=False)
    else:
        print("Did not receieve OK response from Coinbase API")


if __name__ == "__main__":
# we set which pair we want to retrieve data for
    pair = "BTC/USD"
    fetch_daily_data(symbol=pair)

IndentationError: unexpected indent (<ipython-input-56-40ed308d16f6>, line 17)

Selects the dataframe for training

In [61]:
df = yf.download(tickers="LTC-USD",
                            period="60d",
                            interval="15m",
                            auto_adjust=True)

[*********************100%***********************]  1 of 1 completed


In [62]:
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/USDGBP.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/USDJPY.csv')
#df = pd.read_csv('/Users/esteban/Documents/GitHub/Reinforcement_Learning_Binary_Options_Bot/BTCUSD_1hr.csv')
#df = pd.read_csv('BTCUSD_day.csv')
#df = pd.read_csv('BTCUSD_1hr.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/BTCUSD_1hr.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/BTCUSD_1min.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/Download Data - STOCK_US_XNYS_F (1).csv')

df

,Open,High,Low,Close,Volume
Datetime,,,,,
2022-06-23 00:00:00+00:00,52.136105,52.377525,52.089718,52.377525,1541120
2022-06-23 00:15:00+00:00,52.644714,53.492001,52.644714,53.492001,8260224
2022-06-23 00:30:00+00:00,54.019543,54.197636,53.931980,54.197636,17858368
2022-06-23 00:45:00+00:00,54.046234,54.279503,53.970066,54.279503,4132608
2022-06-23 01:00:00+00:00,54.499378,54.499378,54.127907,54.127907,6084544
...,...,...,...,...,...
2022-08-21 15:30:00+00:00,54.968937,55.005173,54.937172,55.005173,2208928
2022-08-21 15:45:00+00:00,55.003269,55.053417,55.003269,55.027519,2392224
2022-08-21 16:00:00+00:00,55.024773,55.093395,55.003059,55.036358,4288448


Data Cleanup

In [63]:
print(df.dtypes)

#Convert date
#df['Date'] = pd.to_datetime(df['Date'])

#Sort values from past to present
#df.sort_values('Date', ascending=True, inplace=True)

#Set date as index
#df.set_index('Date', inplace=True)

#Converts volume from string to float
#df.drop('Volume', inplace=True, axis=1)
#df['Volume'] = df['Volume'].apply(lambda x: float(x.replace(",", "")))
#df['Volume'] = df['Volume'].astype(float)


#Preparation for hourly df
#df = df.drop(columns=['Symbol'])
#df = df.drop(columns=['Unix Timestamp'])
#df = df.drop(columns=['Volume'])

#Preparation for downloaded df

df = df.reset_index(drop=True)
df = df.drop(columns=['Volume'])

'''
#Preparation for daily dataset
df = df.drop(columns=['Volume USD'])
df = df.rename(columns={'Volume BTC': 'Volume'})
'''

print(df.shape[0])
print(df.dtypes)

df.head()

Open      float64
High      float64
Low       float64
Close     float64
Volume      int64
dtype: object
5731
Open     float64
High     float64
Low      float64
Close    float64
dtype: object


,Open,High,Low,Close
0,52.136105,52.377525,52.089718,52.377525
1,52.644714,53.492001,52.644714,53.492001
2,54.019543,54.197636,53.931980,54.197636
3,54.046234,54.279503,53.970066,54.279503
4,54.499378,54.499378,54.127907,54.127907


Normalization

In [ ]:
'''
df["Open"] = df["Open"] - df["Open"].shift(1)
df["High"] = df["High"] - df["High"].shift(1)
df["Low"] = df["Low"] - df["Low"].shift(1)
df["Close"] = df["Close"] - df["Close"].shift(1)
'''

'\ndf["Open"] = df["Open"] - df["Open"].shift(1)\ndf["High"] = df["High"] - df["High"].shift(1)\ndf["Low"] = df["Low"] - df["Low"].shift(1)\ndf["Close"] = df["Close"] - df["Close"].shift(1)\n'

# 2.1 Add indicators
2.1.1 Install New Dependencies

2.1.3. Calculate SMA, RSI and OBV


In [64]:

#Indicators with TA
#Adds simple moving average
df["sma7"] = SMAIndicator(close=df["Open"], window=7, fillna=True).sma_indicator()
df["sma25"] = SMAIndicator(close=df["Open"], window=25, fillna=True).sma_indicator()
df["sma99"] = SMAIndicator(close=df["Open"], window=99, fillna=True).sma_indicator()

#Adds Bollinger Bands
indicator_bb = BollingerBands(close=df["Open"], window=20, window_dev=2)
df['bb_bbm'] = indicator_bb.bollinger_mavg()
df['bb_bbh'] = indicator_bb.bollinger_hband()
df['bb_bbl'] = indicator_bb.bollinger_lband()

#Adds PSAR Indicator
indicator_psar = PSARIndicator(high=df["Close"], low=df["Low"], close=df["Close"], step=0.02, max_step=2, fillna=True)
df['psar'] = indicator_psar.psar()

# Add Moving Average Convergence Divergence (MACD) indicator
df["MACD"] = macd(close=df["Close"], window_slow=26, window_fast=12, fillna=True)
    
# Add Relative Strength Index (RSI) indicator
df["RSI"] = rsi(close=df["Close"], window=14, fillna=True)

#Indicators using finta
#Adds simple moving average
df["sma40"] = TA.SMA(df, period = 40)

# Add Relative Strength Index (RSI) indicator
#df["OBV"] = TA.OBV(df)

#Drops empty spaces
df.fillna(0, inplace=True)

#Drops empty spaces
df.head(50)


/Users/esteban/miniforge3/envs/tf_m1/lib/python3.8/site-packages/ta/trend.py:938: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_up = pd.Series(index=self._psar.index)
/Users/esteban/miniforge3/envs/tf_m1/lib/python3.8/site-packages/ta/trend.py:939: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_down = pd.Series(index=self._psar.index)


,Open,High,Low,Close,sma7,sma25,sma99,bb_bbm,bb_bbh,bb_bbl,psar,MACD,RSI,sma40
0,52.136105,52.377525,52.089718,52.377525,52.136105,52.136105,52.136105,0.000000,0.000000,0.000000,52.377525,0.000000,100.000000,0.000000
1,52.644714,53.492001,52.644714,53.492001,52.390409,52.390409,52.390409,0.000000,0.000000,0.000000,53.492001,0.088904,100.000000,0.000000
2,54.019543,54.197636,53.931980,54.197636,52.933454,52.933454,52.933454,0.000000,0.000000,0.000000,52.089718,0.213835,100.000000,0.000000
3,54.046234,54.279503,53.970066,54.279503,53.211649,53.211649,53.211649,0.000000,0.000000,0.000000,52.174035,0.315809,100.000000,0.000000
4,54.499378,54.499378,54.127907,54.127907,53.469195,53.469195,53.469195,0.000000,0.000000,0.000000,52.300363,0.380011,91.228892,0.000000
5,54.053062,54.085670,54.010975,54.085670,53.566506,53.566506,53.566506,0.000000,0.000000,0.000000,52.419111,0.422612,88.889580,0.000000
6,54.304455,54.379097,54.213799,54.213799,53.671927,53.671927,53.671927,0.000000,0.000000,0.000000,52.530735,0.461394,89.748378,0.000000
7,54.187977,54.187977,54.081806,54.081806,53.965052,53.736434,53.736434,0.000000,0.000000,0.000000,52.635661,0.475992,82.660024,0.000000
8,53.955078,53.995941,53.947216,53.947216,54.152247,53.760727,53.760727,0.000000,0.000000,0.000000,52.734291,0.471267,76.063093,0.000000
9,53.925339,54.117023,53.925339,54.117023,54.138789,53.777188,53.777188,0.000000,0.000000,0.000000,52.827004,0.475741,78.404796,0.000000


In [65]:
#add a column to dataframe
df.drop(df.tail(1).index,inplace=True)
df['Position'] = np.where((df['Open']-df['Close'].shift(-1)) < 1, 1, 0)

#df["Position"] = df['Open'] - df['Close'].shift(-1)



In [66]:
df = df.reset_index(drop=True)


Dump dataset into a pickle file

In [53]:
import pickle
with open("BTCUSD_15min_tagged.pickle", 'wb') as f:
      pickle.dump(df,f, pickle.HIGHEST_PROTOCOL)
